## Implicit collaborative filtering approaches - pseudo product index

### Background

This is the development pipeline to apply the implicit item-based recommendation system algorithms for gracioustyle customers. The aim is to test and compare a few algorithms including  
  
* Alternating Least Square
* Bayesian Personalized Ranking
  
The approaches above can be built into a hybrid CF approach, and further combined with the content based apporach as the optimized strategy. 

We modified the modelling level to a pseudo product index concatenating product brand, type and color, instead of the actual product level. 

### Import libraries

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import random
import implicit
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.movielens import get_movielens
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight,tfidf_weight)
import tqdm
import codecs
from tabulate import tabulate
from ipywidgets import FloatProgress

### Import data

Order data has been merged with product table to build a master source data in json/csv format.

In [3]:
retail_data = pd.read_json('C:/Projects/DJ_projects/FP-growth/recommendation_engine/development/hybrid/data/customer_data_20201228.json') # This may take a couple minutes

In [4]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143173 entries, 0 to 143172
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CUSTOMER_DIM_KEY   143173 non-null  int64  
 1   PRODUCT_DIM_KEY    143173 non-null  int64  
 2   QUANTITY           143172 non-null  float64
 3   PRODUCT_NAME       143166 non-null  object 
 4   COLOR              143173 non-null  object 
 5   PRODUCT_TYPE       143173 non-null  object 
 6   CATEGORY           143173 non-null  object 
 7   BRAND              143173 non-null  object 
 8   COLLECTION_ID      143173 non-null  object 
 9   RESOLUTION_STATUS  143173 non-null  object 
 10  ORDER_DATE_KEY     143173 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 12.0+ MB


In [23]:
retail_data.shape

(143173, 13)

In [6]:
retail_data.count()

CUSTOMER_DIM_KEY     143173
PRODUCT_DIM_KEY      143173
QUANTITY             143172
PRODUCT_NAME         143166
COLOR                143173
PRODUCT_TYPE         143173
CATEGORY             143173
BRAND                143173
COLLECTION_ID        143173
RESOLUTION_STATUS    143173
ORDER_DATE_KEY       143173
dtype: int64

Build a pseudo product index combining the product type, brand and color

In [8]:
retail_data.groupby('COLLECTION_ID').count()

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,QUANTITY,PRODUCT_NAME,COLOR,PRODUCT_TYPE,CATEGORY,BRAND,RESOLUTION_STATUS,ORDER_DATE_KEY,PROD_INDEX
COLLECTION_ID,,,,,,,,,,,
11301,5,5,5,5,5,5,5,5,5,5,5
3-IN-1-DOWN,2,2,2,2,2,2,2,2,2,2,2
ABB12502,4,4,4,4,4,4,4,4,4,4,4
ABB12503,16,16,16,16,16,16,16,16,16,16,16
ABBAL-FRESCO,4,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...
foliagedinner,14,14,14,14,14,14,14,14,14,14,14
incantoturtdin,31,31,31,31,31,31,31,31,31,31,31
simplyannagold,155,155,155,155,155,155,155,155,155,155,155


In [9]:
prod_cols_old = ['PRODUCT_TYPE', 'BRAND', 'COLOR']
retail_data['PROD_INDEX_old'] = retail_data[prod_cols_old].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

prod_cols = ['COLLECTION_ID', 'PRODUCT_TYPE', 'BRAND', 'COLOR']
retail_data['PROD_INDEX'] = retail_data[prod_cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [ ]:
retail_data.head()

In [10]:
retail_data.groupby('PROD_INDEX')['QUANTITY'].agg('sum').sort_values(ascending=False)

PROD_INDEX
NO_COLLECTION_bath tumblers,drinking glasses_Mario Luca Giusti_Unknown    3295.0
FELT-TABLE-LINER_table protectors_Gracious Style_White                    2989.0
NO_COLLECTION_placemats_Kim Seybert_Unknown                               2816.0
NO_COLLECTION_Unknown_Gracious Style_Unknown                              2673.0
NO_COLLECTION_drinking glasses,bath tumblers_Mario Luca Giusti_Unknown    2478.0
                                                                           ...  
POMCROWN-DINNERWARE_serving bowls_Pom Pom at Home_Unknown                   -4.0
POMCROWN-DINNERWARE_salad & dessert plates_Pom Pom at Home_Unknown          -6.0
POMCROWN-DINNERWARE_dinner plates_Pom Pom at Home_Unknown                  -12.0
POMCROWN-DINNERWARE_cereal & soup bowls_Pom Pom at Home_Unknown            -12.0
NO_COLLECTION_Unknown_Anali_Cream                                          -15.0
Name: QUANTITY, Length: 26018, dtype: float64

In [12]:
retail_data.groupby('PROD_INDEX_old')['QUANTITY'].agg('sum').sort_values(ascending=False)

PROD_INDEX_old
napkins_Garnier-Thiebaut_Unknown                            4085.0
table protectors_Gracious Style_White                       3538.0
bath tumblers,drinking glasses_Mario Luca Giusti_Unknown    3327.0
placemats_Kim Seybert_Unknown                               2818.0
dinner plates_Le Cadeaux_Unknown                            2806.0
                                                             ...  
serving bowls_Pom Pom at Home_Unknown                         -4.0
salad & dessert plates_Pom Pom at Home_Unknown                -6.0
cereal & soup bowls_Pom Pom at Home_Unknown                   -8.0
dinner plates_Pom Pom at Home_Unknown                         -8.0
Unknown_Anali_Cream                                          -15.0
Name: QUANTITY, Length: 9501, dtype: float64

In [14]:
retail_data['PROD_INDEX_old'].value_counts()

table protectors_Gracious Style_White       3345
tablecloths_Sferra_White                    1570
detergents_Le Blanc_Unknown                 1520
rugs_Dash & Albert_Unknown                  1483
soap dispensers_Kassatex_Unknown            1389
                                            ... 
dining tables_Interlude Home_Unknown           1
baskets_Selamat_Unknown                        1
coffee & tea pots_Chelsea House_Unknown        1
buffet & chargers_Niderviller_Yellow           1
bath sheets_Gracious Style_White/Lt Blue       1
Name: PROD_INDEX_old, Length: 9501, dtype: int64

In [17]:
retail_data.groupby('PROD_INDEX_old')['COLLECTION_ID'].nunique().sort_values(ascending=False)

PROD_INDEX_old
rugs_Dash & Albert_Unknown                     322
tablecloths_Garnier-Thiebaut_Unknown           155
tissue boxes_Pigeon & Poodle_Unknown           148
waste baskets_Pigeon & Poodle_Unknown          133
shams_Pine Cone Hill_Unknown                   130
                                              ... 
pillow cases,bedding_Sferra_White / Salmon       1
pillow cases,bedding_Sferra_White / Taupe        1
pillow cases,bedding_Sferra_White / White        1
pillow cases,bedding_Sferra_White/Aubergine      1
hand towels_Kassatex_Skylight Blue               1
Name: COLLECTION_ID, Length: 9501, dtype: int64

In [36]:
retail_data[retail_data.PROD_INDEX_old == 'tablecloths_Garnier-Thiebaut_Unknown'].sample(10)

,CUSTOMER_DIM_KEY,PRODUCT_DIM_KEY,QUANTITY,PRODUCT_NAME,COLOR,PRODUCT_TYPE,CATEGORY,BRAND,COLLECTION_ID,RESOLUTION_STATUS,ORDER_DATE_KEY,PROD_INDEX,PROD_INDEX_old
20067,69571,134333,1.0,"Eloise Diamant Tablecloth 69"" x 69"" Green Swee...",Unknown,tablecloths,ELOISE-DIAMANT,Garnier-Thiebaut,ELOISE-DIAMANT,COMPLETED,18371,ELOISE-DIAMANT_tablecloths_Garnier-Thiebaut_Un...,tablecloths_Garnier-Thiebaut_Unknown
86238,54636,333293,0.0,"Round Around 64 in square Tablecloth, Cappuccino",Unknown,tablecloths,Unknown,Garnier-Thiebaut,GTHRND-CAPPUCCINO,COMPLETED,16678,GTHRND-CAPPUCCINO_tablecloths_Garnier-Thiebaut...,tablecloths_Garnier-Thiebaut_Unknown
69915,46535,186569,1.0,"Mille Datcha Aubergine Tablecloth 93"" Round, 1...",Unknown,tablecloths,Unknown,Garnier-Thiebaut,MDATCHA-AUBERGINE,COMPLETED,16017,MDATCHA-AUBERGINE_tablecloths_Garnier-Thiebaut...,tablecloths_Garnier-Thiebaut_Unknown
123505,34885,104583,1.0,Eloise Parchemin Tablecloth 69 x 120,Unknown,tablecloths,Unknown,Garnier-Thiebaut,NO_COLLECTION,COMPLETED,15243,NO_COLLECTION_tablecloths_Garnier-Thiebaut_Unk...,tablecloths_Garnier-Thiebaut_Unknown
38415,87049,472794,1.0,"Mille Wax Ketchup Tablecloth 45"" x 45"" 100% Co...",Unknown,tablecloths,Unknown,Garnier-Thiebaut,MWAX-KETCHUP,COMPLETED,18529,MWAX-KETCHUP_tablecloths_Garnier-Thiebaut_Unknown,tablecloths_Garnier-Thiebaut_Unknown
61840,36395,134272,0.0,Comtesse Griotte Tablecloth Rect 69 x 120 in,Unknown,tablecloths,Unknown,Garnier-Thiebaut,NO_COLLECTION,COMPLETED,15676,NO_COLLECTION_tablecloths_Garnier-Thiebaut_Unk...,tablecloths_Garnier-Thiebaut_Unknown
94566,58028,333247,1.0,"Natte 90 in square Tablecloth, Ivory",Unknown,tablecloths,Unknown,Garnier-Thiebaut,GTHNATTE-IVORY,COMPLETED,16989,GTHNATTE-IVORY_tablecloths_Garnier-Thiebaut_Un...,tablecloths_Garnier-Thiebaut_Unknown
51599,94068,134275,1.0,"Isaphire Emeraude Tablecloth 69"" x 100"" Green ...",Unknown,tablecloths,ISAPHIRE-EMERAUDE,Garnier-Thiebaut,ISAPHIRE-EMERAUDE,PENDING,18617,ISAPHIRE-EMERAUDE_tablecloths_Garnier-Thiebaut...,tablecloths_Garnier-Thiebaut_Unknown
24744,81314,523661,0.0,"Veneziano Sfumato Tablecloth 69"" x 120"" Green ...",Unknown,tablecloths,Unknown,Garnier-Thiebaut,GTH16695,CANCELLED,18412,GTH16695_tablecloths_Garnier-Thiebaut_Unknown,tablecloths_Garnier-Thiebaut_Unknown
60740,37026,134348,1.0,"Beauregard Ivoire Tablecloth 75"" x 146"" 100% C...",Unknown,tablecloths,BEAUREGARD-IVORY,Garnier-Thiebaut,BEAUREGARD-IVORY,COMPLETED,15647,BEAUREGARD-IVORY_tablecloths_Garnier-Thiebaut_...,tablecloths_Garnier-Thiebaut_Unknown


In [21]:
retail_data['COLLECTION_ID'].value_counts().sort_values(ascending=False)

NO_COLLECTION           67829
FELT-TABLE-LINER         2885
POINT-DINNERWARE          552
LE-BAIN-BATH-ACCESS       462
VIZCAYA-BATH-ACCESSO      461
                        ...  
MLIVIENNA                   1
JPRMOY01                    1
ABBLANDSCAPE-PAES           1
PCHDALM-FLAN-PAJA           1
SRAWAT5004                  1
Name: COLLECTION_ID, Length: 6498, dtype: int64

In [24]:
67829/143173

0.4737555265308403

Some data subseting has been done based on the following conditions,  
  
* Only contains the order history on products that have had orders since 2018. This is used as a proxy as the current product catelogue. 
* Resolution status in 'Completed' or 'Pending'.
* De-duplicate the identical rows.

In [ ]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.PRODUCT_NAME) == False]
cleaned_retail = cleaned_retail[cleaned_retail['PRODUCT_DIM_KEY'].isin(cleaned_retail[cleaned_retail['ORDER_DATE_KEY'] > 17533].PRODUCT_DIM_KEY)]
cleaned_retail['QUANTITY'] = cleaned_retail['QUANTITY'].fillna(0.0)
cleaned_retail = cleaned_retail[cleaned_retail.RESOLUTION_STATUS.isin(['COMPLETED', 'PENDING'])]
print('Duplicated rows: ' + str(cleaned_retail.duplicated().sum()))
cleaned_retail.drop_duplicates(inplace=True)

In [ ]:
cleaned_retail.info()

In [ ]:
cleaned_retail['PROD_INDEX'].value_counts()

### Threhold function for cut-off

In [ ]:
def threshold_likes(df, uid_min, mid_min, user_column = 'CUSTOMER_DIM_KEY', item_column = 'PROD_INDEX'):
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby(user_column)[item_column].count()
        df = df[~df[user_column].isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby(item_column)[user_column].count()
        df = df[~df[item_column].isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby(user_column)[item_column].count().min() >= mid_min)
    assert(df.groupby(item_column)[user_column].count().min() >= uid_min)
    
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

### Product lookup table

In [ ]:
df_lim = threshold_likes(cleaned_retail, 2, 2)
item_lookup = df_lim[['PROD_INDEX']].drop_duplicates() # Only get unique item/description pairs
item_lookup['PROD_INDEX'] = item_lookup.PROD_INDEX.astype(str) # Encode as strings for future lookup ease

### User-product matrix function

In [ ]:
def create_matrix(cleaned_retail, row_feature, col_feature):
    # cleaned_retail[row_feature] = cleaned_retail[row_feature].astype(int) # Convert to int for customer ID
    cleaned_retail = cleaned_retail[[col_feature, 'QUANTITY', row_feature]] # Get rid of unnecessary info
    grouped_cleaned = cleaned_retail.groupby([row_feature, col_feature]).sum().reset_index() # Group together
    grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
    # indicate purchased
    grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive
    
    customers = list(np.sort(grouped_purchased[row_feature].unique())) # Get our unique customers
    products = list(np.sort(grouped_purchased[col_feature].unique())) # Get our unique products that were purchased
    quantity = list(grouped_purchased.QUANTITY) # All of our purchases
    #print(products)
    rows = grouped_purchased[row_feature].astype('category', CategoricalDtype(categories = customers)).cat.codes 
    # Get the associated row indices
    cols = grouped_purchased[col_feature].astype('category', CategoricalDtype(categories = products)).cat.codes 
    # Get the associated column indices
    purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
    return purchases_sparse, customers, products

### Create user-product matrix

In [ ]:
purchases_sparse, products, customers = create_matrix(df_lim, 'PROD_INDEX', 'CUSTOMER_DIM_KEY')

In [ ]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [ ]:
products_arr

In [ ]:
len(set(customers_arr))

In [ ]:
len(set(products_arr))

### Train and test data prep

In [ ]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [ ]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

### Function to calculate AUC (evaluation metric)

In [ ]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
def calc_mean_auc_cv(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
def calc_mean_auc_pop(training_set, altered_users, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    
    return float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1],
              'alpha': [1, 10, 50, 100]}

def cross_validation_als(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9)).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)

    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for alpha in param_grid['alpha']:
                model = AlternatingLeastSquares(factors = factor, regularization = regu)
                model.fit(training_set*alpha)
                item_vecs = model.user_factors
                user_vecs = model.item_factors
                store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                print(factor, regu, alpha, store_auc)
                
                if store_auc > best_auc:
                    best_auc = store_auc
                    best_factors = factor
                    best_reg = regu
                    best_alpha = alpha
                    best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best factor: ', best_factors)
    print('Best regularization: ', best_reg)
    print('Best alpha: ', best_alpha)
    return best_auc, pop_auc, best_factors, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

### ALS hyper tuning with cross validation

In [ ]:
cv_als = cross_validation_als(purchases_sparse, 'als', param_grid)

In [ ]:
cv_als[4]

### ALS hyper tuning distance parameters

In [ ]:
model = AlternatingLeastSquares(factors = cv_als[2], regularization = cv_als[3])
# The literature suggests values between 0 and 3 for K1 and 0 and 1 for B. 
ratings = bm25_weight(product_train, K1 = 0.1, B = 1).tocsr()
model.fit(ratings)
item_vecs = model.user_factors
user_vecs = model.item_factors
store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
store_auc, pop_auc

In [ ]:
param_grid = {'B': [0.1, 0.5, 1, 2, 3],
              'K1': [0, 0.1, 0.01, 1]
             }

def cross_validation_als_bm25_param(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9)).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)
    best_auc=0
    best_B = 0
    best_K1 = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for B in param_grid['B']:
        for K1 in param_grid['K1']:
            training_set = (bm25_weight(product_train, B=B,K1=K1)).tocsr()
            model = AlternatingLeastSquares(factors = cv_als[2], regularization = cv_als[3])
            model.fit(training_set)
            item_vecs = model.user_factors
            user_vecs = model.item_factors
            store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
            print(B, K1, store_auc)
                
            if store_auc > best_auc:
                best_auc = store_auc
                best_B = B
                best_K1 = K1
                best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best B: ', best_B)
    print('Best K1: ', best_K1)
    return best_auc, pop_auc, best_B, best_K1, best_model
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [ ]:
cv_als_w = cross_validation_als_bm25_param(purchases_sparse, 'als', param_grid)

### Best ALS model

Select the best ALS model from the hyper tuning process.

In [ ]:
best_als = cv_als_w[4]

### BPR-MF hyper tuning with cross validation

In [ ]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001]
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [0.1],
              'regularization': [0.0, 0.1,0.01,0.001],
              'alpha': [1, 10, 50, 100, 500]}

def cross_validation_bpr(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    if model_name == "bpr":
        model = BayesianPersonalizedRanking()
    if model_name == "lmf":
        model = LogisticMatrixFactorization()
        
    training_set = product_train
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_learning_rate = 0

    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for lr in param_grid['learning_rate']:
                for alpha in param_grid['alpha']:
                    model = BayesianPersonalizedRanking(factors = factor,learning_rate= lr, regularization = regu)
                    model.fit(training_set*alpha)
                    item_vecs = model.user_factors
                    user_vecs = model.item_factors
                    store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                    print(factor, regu, lr, alpha, store_auc, pop_auc)

                    if store_auc > best_auc:
                        best_auc = store_auc
                        best_factors = factor
                        best_reg = regu
                        best_alpha = alpha
                        best_learning_rate = lr
                        best_model = model
    print(best_auc, best_factors, best_learning_rate, best_reg, best_alpha)
    return best_auc, best_factors,best_learning_rate, best_reg, best_alpha, best_model
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [ ]:
cv_bpr = cross_validation_bpr(purchases_sparse, 'bpr', param_grid)

### Best BPR-MF model

Select the best BPR model from the hyper tuning process.

In [ ]:
best_bpr = cv_bpr[5]

In [ ]:
customers_arr[5000:5100]

### Get actual purchase vs model recommendations

In [ ]:
def get_rec_items(model, customer_id, sparse_matrix, customer_list, item_list, item_lookup, num_items = 10):
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = sparse_matrix[:,cust_ind].nonzero()[0] # Get column indices of purchased items
    prod_codes = item_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    purchased_items = item_lookup.loc[item_lookup.PROD_INDEX.isin(prod_codes)]
    
    print('-------------------------------------------------------------------------------------')
    print('Print purchased items in train set')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(purchased_items))
    print('-------------------------------------------------------------------------------------')
    print('Print full actual purchases above threshold')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(df_lim[df_lim['CUSTOMER_DIM_KEY'] == customer_id], tablefmt="tsv"))
    print('-------------------------------------------------------------------------------------')
    print('Print model recommendations')
    print('-------------------------------------------------------------------------------------')
    
    recom = model.recommend(cust_ind, sparse_matrix.T.tocsr(),filter_items=purchased_ind.tolist(), N= num_items)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in recom:
        key = item_list[index[0]]
        rec_keys.append(key)
        # descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in recom]
    final_frame = pd.DataFrame({'PROD_INDEX': rec_keys, 'SCORE': scores}) # Create a dataframe 
    final_frame[['PROD_INDEX','SCORE']]
    print(tabulate(final_frame, tablefmt="tsv"))
    return final_frame[['PROD_INDEX','SCORE']] # Switch order of columns around

In [ ]:
get_rec_items(best_als, 63356, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 63356, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 73183, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 73183, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 69065, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 69065, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 64522, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 64522, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 67130, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 67130, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 70154, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 70154, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 70731, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 70731, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 64041, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 64041, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 64489, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 64489, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_als, 68625, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

In [ ]:
get_rec_items(best_bpr, 68625, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

### Product index similarities

In [ ]:
def get_sim_item(model, item_id):
    purchased_items = item_lookup.loc[item_lookup.PROD_INDEX.isin([item_id])]
    item_ind = np.where(products_arr == item_id)
    print(purchased_items)
    arr = model.similar_items(item_ind[0][0], N=20)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in arr:
        key = products_arr[index[0]]
        rec_keys.append(key)
        # descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in arr]
    final_frame = pd.DataFrame({'PROD_INDEX': rec_keys, 'SCORE': scores}) # Create a dataframe 
    final_frame[['PROD_INDEX', 'SCORE']]
    return final_frame[['PROD_INDEX','SCORE']] # Switch order of columns around

In [ ]:
item_lookup.loc[item_lookup.PROD_INDEX.isin(['blankets_Pine Cone Hill_Unknown'])]

In [ ]:
# Best ALS
get_sim_item(model = best_als, item_id = 'blankets_Pine Cone Hill_Unknown')

In [ ]:
# Best BPR-MF to rank similar product indexes
get_sim_item(model = best_bpr, item_id = 'blankets_Pine Cone Hill_Unknown')

In [ ]:
# Best BPR-MF to rank similar product indexes
get_sim_item(model = best_bpr, item_id = 'table protectors_Gracious Style_White')

In [ ]:
# Best BPR-MF to rank similar product indexes
get_sim_item(model = best_bpr, item_id = 'detergents_Le Blanc_Unknown')

In [ ]:
# Best BPR-MF to rank similar product indexes
get_sim_item(model = best_bpr, item_id = 'salad & dessert plates_Anna Weatherley_Unknown')